vol clustering

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import seaborn as sns

prices = np.loadtxt("prices.txt")
prices = prices.T

In [3]:
log_returns = np.log(prices[:, 1:] / prices[:, :-1])
squared_returns = log_returns ** 2

from statsmodels.tsa.stattools import acf

volatility_clustered = []

for i, sq_ret in enumerate(squared_returns):
    acf_vals = acf(sq_ret, nlags=10, fft=True)
    if np.any(acf_vals[1:] > 0.3):
        volatility_clustered.append({
            'Instrument': i,
            'ACF Lag 1': acf_vals[1],
            'ACF Lag 2': acf_vals[2],
            'ACF Lag 3': acf_vals[3]
        })

vol_cluster_df = pd.DataFrame(volatility_clustered)
if not vol_cluster_df.empty:
    print(vol_cluster_df.sort_values(by='ACF Lag 1', ascending=False))
else:
    print("No instruments exhibit significant volatility clustering based on the ACF threshold.")


No instruments exhibit significant volatility clustering based on the ACF threshold.


In [4]:
log_returns = np.log(prices[:, 1:] / prices[:, :-1])
squared_returns = log_returns ** 2

volatility_clustered = []

for i, sq_ret in enumerate(squared_returns):
    acf_vals = acf(sq_ret, nlags=10, fft=True)
    if np.any(acf_vals[1:] > 0.2):
        volatility_clustered.append({
            'Instrument': i,
            'ACF Lag 1': acf_vals[1],
            'ACF Lag 2': acf_vals[2],
            'ACF Lag 3': acf_vals[3]
        })

vol_cluster_df = pd.DataFrame(volatility_clustered)

if not vol_cluster_df.empty:
    print(vol_cluster_df.sort_values(by='ACF Lag 1', ascending=False))
else:
    print("No instruments exhibit significant volatility clustering above the relaxed 0.2 threshold.")


No instruments exhibit significant volatility clustering above the relaxed 0.2 threshold.


In [5]:
from arch import arch_model

garch_results = []

for i, y in enumerate(log_returns):
    model = arch_model(y * 100, vol='GARCH', p=1, q=1, dist='normal')
    try:
        res = model.fit(disp='off')
        pval_alpha = res.pvalues['alpha[1]']
        pval_beta = res.pvalues['beta[1]']
        sig = (pval_alpha < 0.05) and (pval_beta < 0.05)
        garch_results.append({
            'Instrument': i,
            'alpha[1]': res.params['alpha[1]'],
            'beta[1]': res.params['beta[1]'],
            'p(alpha)': pval_alpha,
            'p(beta)': pval_beta,
            'Significant': sig
        })
    except:
        continue

garch_df = pd.DataFrame(garch_results)
significant_garch = garch_df[garch_df['Significant']]

print(significant_garch.sort_values(by='p(alpha)'))


Empty DataFrame
Columns: [Instrument, alpha[1], beta[1], p(alpha), p(beta), Significant]
Index: []
